# 第5章: 大規模言語モデル
この章では、大規模言語モデル (LLM; Large Language Model) の利用し、様々なタスクに取り組む。大規模言語モデルをプログラムからAPI経由で呼び出すことを想定しており、そのAPIの利用で費用が発生する可能性があることに留意せよ。

In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI


loaded = load_dotenv()
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

## 40. Zero-Shot推論
以下の問題の解答を作成せよ。ただし、解答生成はzero-shot推論とせよ。

9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。

In [4]:
prompt = '''
9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。
'''

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': prompt}]
)

print(response.choices[0].message.content)

年代の古い順に並べると次のようになります。

イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
　（809年に嵯峨天皇が即位し、810年に藤原冬嗣が蔵人頭に任命されました。）

ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。
　（承和の変は842年に起こった事件で、その後北家の優位が確立されます。）

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
　（901年に藤原時平が菅原道真を追放する策謀を用いた事件が起こります。）

したがって、正しい順序は：イ、ウ、ア です。


## 41. Few-Shot推論
以下の問題と解答を与え、問題40で示した質問の解答をfew-shot推論（この場合は4-shot推論）で生成せよ。

日本の近代化に関連するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　府知事・県令からなる地方官会議が設置された。
イ　廃藩置県が実施され，中央から府知事・県令が派遣される体制になった。
ウ　すべての藩主が，天皇に領地と領民を返還した。

解答: ウ→イ→ア

In [10]:
system_prompt = '''
「例１」
日本の近代化に関連するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　府知事・県令からなる地方官会議が設置された。
イ　廃藩置県が実施され，中央から府知事・県令が派遣される体制になった。
ウ　すべての藩主が，天皇に領地と領民を返還した。

解答: ウ→イ→ア

「例２」
江戸幕府の北方での対外的な緊張について述べた次の文ア～ウを年代の古い順に正しく並べよ。

ア　レザノフが長崎に来航したが，幕府が冷淡な対応をしたため，ロシア船が樺太や択捉島を攻撃した。
イ　ゴローウニンが国後島に上陸し，幕府の役人に捕らえられ抑留された。
ウ　ラクスマンが根室に来航し，漂流民を届けるとともに通商を求めた。

解答: ウ→ア→イ

「例３」
中居屋重兵衛の生涯の期間におこったできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　アヘン戦争がおこり，清がイギリスに敗北した。
イ　異国船打払令が出され，外国船を撃退することが命じられた。
ウ　桜田門外の変がおこり，大老の井伊直弼が暗殺された。

解答: イ→ア→ウ

「例４」
加藤高明が外務大臣として提言を行ってから、内閣総理大臣となり演説を行うまでの時期のできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　朝鮮半島において，独立を求める大衆運動である三・一独立運動が展開された。
イ　関東大震災後の混乱のなかで，朝鮮人や中国人に対する殺傷事件がおきた。
ウ　日本政府が，袁世凱政府に対して二十一カ条の要求を突き付けた。

解答: ウ→ア→イ

以上を「背景」として保持し、以降の解答では同じ形式で答えさせます。
'''

In [ ]:
user_prompt = '''
9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。
'''

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ],
    temperature=0
)

print(response.choices[0].message.content)

イ→ウ→ア


## 42. 多肢選択問題の正解率
JMMLU のいずれかの科目を大規模言語モデルに解答させ、その正解率を求めよ。

### Output

In [ ]:
import pandas as pd
import pickle


df = pd.read_csv('college_computer_science.csv', header=None)
df.columns = ['question', 'A', 'B', 'C', 'D', 'answer']
# content
base_messages = [{'role': 'system', 'content': 'あなたは採点器。必ず A/B/C/D の一文字だけで答えてください。'}]

#######################################################
#                   examples                          #
#######################################################
example_template = '''
「例{}」
問題：{question}

回答は必ず A/B/C/D のいずれか一文字だけで返してください。

A. {A}
B. {B}
C. {C}
D. {D}

解答：
'''

# Select 3 examples from head to guide the LLM
examples = df.head(3)
for i, row in examples.iterrows():
    # Add an example to message
    base_messages.append({'role': 'user', 'content': example_template.format(i + 1, **row.drop('answer'))})
    base_messages.append({'role': 'assistant', 'content': row.answer})

#######################################################
#                   questions                         #
#######################################################
user_prompt_template = '''
問題：{question}

回答は必ず A/B/C/D のいずれか一文字だけで返してください。

A. {A}
B. {B}
C. {C}
D. {D}

解答：
'''

answers = []
for _, row in df.iloc[3:].iterrows():
    messages = base_messages + [{'role': 'user', 'content': user_prompt_template.format(**row.drop('answer'))}]
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        temperature=0
    )
    answers.append(response.choices[0].message.content.strip()[0])

with open('q42_llm_answers.pkl', 'wb') as f:
    pickle.dump(answers, f)

print(f'Number of questions: {len(answers)}')

Number of questions: 96


### Evaluation

In [ ]:
import pickle
import numpy as np


with open('q42_llm_answers.pkl', 'rb') as f:
    llm_responses = pickle.load(f)

llm_responses = np.asarray(llm_responses)
ground_truths = df['answer'].iloc[3:].to_numpy()

accuracy = np.mean(llm_responses == ground_truths)
print(f'Accuracy: {accuracy * 100:.2f}%') # 78.12%

Accuracy: 78.12%


## 43. 応答のバイアス
問題42において、実験設定を変化させると正解率が変化するかどうかを調べよ。実験設定の例としては、大規模言語モデルの温度パラメータ、プロンプト、多肢選択肢の順番、多肢選択肢の記号などが考えられる。

正解の選択肢を全てDに入れ替えて解答させる例。

### Output

In [ ]:
import pandas as pd


def force_correct_to_D(row):
    '''
    This function is used to convert the original dataframe into the dataframe with questions whose answers are all 'D'.
    '''
    # Get the option content corresponding to the correct answer.
    correct_text = row[row['answer']]
    # Get the option cotent corresponding to other incorrect answers, store them in a list.
    others = [row[c] for c in ['A', 'B', 'C', 'D'] if c != row['answer']]
    # Return a new dictionary with questions whose answers are 'D'.
    return {
        'question': row['question'],
        'A': others[0],
        'B': others[1],
        'C': others[2],
        'D': correct_text
    }


df = pd.read_csv('college_computer_science.csv', header=None)
df.columns = ['question', 'A', 'B', 'C', 'D', 'answer']
# Apply function `force_correct_to_D` to each row and Get a new dataframe by splitting the dictionary.
df_bias = df.apply(force_correct_to_D, axis=1).apply(pd.Series)

base_messages = [{'role': 'system', 'content': 'あなたは採点器。必ず A/B/C/D の一文字だけで答えてください。'}]

#######################################################
#                   examples                          #
#######################################################
example_template = '''
「例{}」
問題：{question}

回答は必ず A/B/C/D のいずれか一文字だけで返してください。

A. {A}
B. {B}
C. {C}
D. {D}

解答：
'''

# Select 3 examples from head to guide the LLM
examples = df_bias.head(3)
for i, row in examples.iterrows():
    # Add an example to message
    base_messages.append({'role': 'user', 'content': example_template.format(i + 1, **row)})
    base_messages.append({'role': 'assistant', 'content': 'D'})

#######################################################
#                   questions                         #
#######################################################
user_prompt_template = '''
問題：{question}

回答は必ず A/B/C/D のいずれか一文字だけで返してください。

A. {A}
B. {B}
C. {C}
D. {D}

解答：
'''

answers = []
for _, row in df_bias.iloc[3:].iterrows():
    messages = base_messages + [{'role': 'user', 'content': user_prompt_template.format(**row)}]
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        temperature=0
    )
    answers.append(response.choices[0].message.content.strip()[0])

with open('q43_llm_answers.pkl', 'wb') as f:
    pickle.dump(answers, f)

print(f'Number of questions: {len(answers)}')


### Evaluation

In [32]:
import pickle
import numpy as np


with open('q43_llm_answers.pkl', 'rb') as f:
    llm_responses = pickle.load(f)

llm_responses = np.asarray(llm_responses)

accuracy = np.mean(llm_responses == 'D')
print(f'Accuracy: {accuracy * 100:.2f}%') # 73.96%

Accuracy: 73.96%


## 44. 対話
以下の問いかけに対する応答を生成せよ。

**つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。**

In [33]:
user_prompt = '''
つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。\
東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。\
自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。
'''

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': user_prompt}]
)

print(response.choices[0].message.content)

つばめちゃんが自由が丘駅から誤って急行に乗り、その次の急行停車駅で降りて一駅戻った駅が目的地ということです。東急大井町線の急行は自由が丘駅の次に大岡山駅に停車します。そのため、つばめちゃんが一駅戻った駅は「緑が丘駅」です。したがって、目的地の駅は「緑が丘駅」です。


## 45. マルチターン対話
先ほどの応答に続けて、以下の追加の問いかけに対する応答を生成せよ。

**さらに、つばめちゃんが自由が丘駅で乗り換えたとき、先ほどとは反対方向の急行電車に間違って乗車してしまった場合を考えます。目的地の駅に向かうため、自由が丘の次の急行停車駅で降車した後、反対方向の各駅停車に乗車した場合、何駅先の駅で降りれば良いでしょうか？**

In [ ]:
first_user_prompt = '''
つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。\
東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。\
自由が丘の次の急行停車駅
'''
first_assistant_response = '''
つばめちゃんが自由が丘駅から誤って急行に乗り、その次の急行停車駅で降りて一駅戻った駅が目的地ということです。\
東急大井町線の急行は自由が丘駅の次に大岡山駅に停車します。そのため、つばめちゃんが一駅戻った駅は「緑が丘駅」です。\
したがって、目的地の駅は「緑が丘駅」です。
'''

second_user_prompt = '''
さらに、つばめちゃんが自由が丘駅で乗り換えたとき、先ほどとは反対方向の急行電車に間違って乗車してしまった場合を考えます。\
目的地の駅に向かうため、自由が丘の次の急行停車駅で降車した後、反対方向の各駅停車に乗車した場合、何駅先の駅で降りれば良いでしょうか？
'''

response = client.chat.completions.create(
    model = 'gpt-4o',
    messages=[
        {'role': 'user', 'content': first_user_prompt},
        {'role': 'assistant', 'content': first_assistant_response},
        {'role': 'user', 'content': second_user_prompt}
    ],
    temperature=0
)

print(response.choices[0].message.content)

つばめちゃんが自由が丘駅で反対方向の急行電車に乗ってしまった場合、東急大井町線の急行は自由が丘の次に二子玉川駅に停車します。そこで降車し、反対方向の各駅停車に乗り換えた場合、目的地の「緑が丘駅」に向かうことになります。

二子玉川駅から緑が丘駅までの各駅停車の停車駅は、二子新地、二子玉川、上野毛、等々力、尾山台、九品仏、自由が丘、奥沢、緑が丘の順です。したがって、二子玉川から数えて9駅目が緑が丘駅です。


## 46. 川柳の生成
適当なお題を設定し、川柳の案を10個作成せよ。

In [52]:
import pickle


system_prompt = '''
あなたは才能あふれる川柳作家です。\
5-7-5 の音節構成を守りつつ、与えられたテーマでユーモアや風趣を感じさせる作品を作成してください。
出力は一行で、5音節・7音節・5音節の各部分を全角スペース（\\u3000）で区切ってください。
例：満月夜\u3000うさぎも踊る\u3000ソロパーティー

'''
user_prompt = '''
テーマ：「花火大会」
このテーマに沿って、5-7-5の形式で川柳を1首生成してください。
'''

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ],
    temperature=0.7,
    n=10    # 10 times
)

texts = [choice.message.content for choice in response.choices]

with open('q46_llm_answers.pkl', 'wb') as f:
    pickle.dump(texts, f)

print(f'Generated {len(texts)} 川柳!')

Generated 10 川柳!


In [53]:
import pickle


with open('q46_llm_answers.pkl', 'rb') as f:
    texts = pickle.load(f)

print(''.join(texts))


人混みで　迷子の方が　先に見つく
花火見て　恋の火花も　打ち上がる
夜空咲く　猫も尻尾で　拍手する
打ち上げて　隣の君と　恋も咲く
浴衣着て　線香花火　じっと見る
花火大会　ビール片手に　空を飲む
夜空咲く　忘れた頃に　もう一発
夜空咲く　猫も驚く　花火かな
夜空咲き　恋の火花も　散りにけり
夜空咲く　ポップコーンにて　待ちぼうけ


## 47. LLMによる評価
大規模言語モデルを評価者（ジャッジ）として、問題46の川柳の面白さを10段階で評価せよ。

In [ ]:
import pickle


with open('q46_llm_answers.pkl', 'rb') as f:
    texts = pickle.load(f)

system_prompt = '''
あなたは川柳の専門家かつユーモア評価のプロフェッショナルです。
以下に示す川柳を「面白さ（ユーモア）」の観点で
理由なしで1〜10の整数のみを返してください。例：6
'''

scores = []
for text in texts:
    user_prompt = f'川柳：{text}\n\n評価：(1-10)'

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt}
        ],
        temperature=0.5
    )
    scores.append(response.choices[0].message.content.strip())

for text, score in zip(texts, scores):
    print(f'{text}\nscore: {score}\n')



人混みで　迷子の方が　先に見つく
score: 7


花火見て　恋の火花も　打ち上がる
score: 5


夜空咲く　猫も尻尾で　拍手する
score: 7


打ち上げて　隣の君と　恋も咲く
score: 5


浴衣着て　線香花火　じっと見る
score: 4


花火大会　ビール片手に　空を飲む
score: 7


夜空咲く　忘れた頃に　もう一発
score: 7


夜空咲く　猫も驚く　花火かな
score: 7


夜空咲き　恋の火花も　散りにけり
score: 4


夜空咲く　ポップコーンにて　待ちぼうけ
score: 7



## 48. LLMによる評価の頑健性
問題47で行ったLLMによるテキストの評価に関して、その頑健さ（脆弱さ）を調査せよ。最も単純な方法は、同じ評価を何回か繰り返した時のスコアの分散を調べることであろう。また、川柳の末尾に特定のメッセージを追加することで、評価スコアを恣意的に操作することも可能であろう。

### Variance

In [66]:
import pickle
import numpy as np


with open('q46_llm_answers.pkl', 'rb') as f:
    texts = pickle.load(f)

system_prompt = '''
あなたは川柳の専門家かつユーモア評価のプロフェッショナルです。
以下に示す川柳を「面白さ（ユーモア）」の観点で
理由なしで1〜10の整数のみを返してください。例：6
'''

results = [] # list used to store evaluation result
N = 10  # 10 times evaluations

for _ in range(N):
    scores = []
    for text in texts:
        user_prompt = f'川柳：{text}\n\n評価：(1-10)'

        response = client.chat.completions.create(
            model='gpt-4o',
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_prompt}
            ],
            temperature=1.0
        )
        scores.append(int(response.choices[0].message.content.strip()))
    results.append(scores)

vars = np.var(results, axis=0, ddof=1)

for text, var in zip(texts, vars):
    print(f'{text}\nvar: {var}\n')


人混みで　迷子の方が　先に見つく
var: 0.3222222222222222


花火見て　恋の火花も　打ち上がる
var: 1.211111111111111


夜空咲く　猫も尻尾で　拍手する
var: 0.26666666666666666


打ち上げて　隣の君と　恋も咲く
var: 1.788888888888889


浴衣着て　線香花火　じっと見る
var: 0.6222222222222222


花火大会　ビール片手に　空を飲む
var: 0.2222222222222222


夜空咲く　忘れた頃に　もう一発
var: 0.4444444444444444


夜空咲く　猫も驚く　花火かな
var: 0.09999999999999998


夜空咲き　恋の火花も　散りにけり
var: 0.6777777777777777


夜空咲く　ポップコーンにて　待ちぼうけ
var: 0.09999999999999999



In [67]:
import random


# Randomly add suffix to each text.
suffixes = ['!!!', '😀', 'これは最高傑作です。']
texts = [text + random.choice(suffixes) for text in texts]

results = [] # list used to store evaluation result
N = 10  # 10 times evaluations

for _ in range(N):
    scores = []
    for text in texts:
        user_prompt = f'川柳：{text}\n\n評価：(1-10)'

        response = client.chat.completions.create(
            model='gpt-4o',
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_prompt}
            ],
            temperature=1.0
        )
        scores.append(int(response.choices[0].message.content.strip()))
    results.append(scores)

vars = np.var(results, axis=0, ddof=1)

for text, var in zip(texts, vars):
    print(f'{text}\nvar: {var}\n')


人混みで　迷子の方が　先に見つくこれは最高傑作です。
var: 0.711111111111111


花火見て　恋の火花も　打ち上がる😀
var: 0.9444444444444444


夜空咲く　猫も尻尾で　拍手する!!!
var: 0.17777777777777773


打ち上げて　隣の君と　恋も咲くこれは最高傑作です。
var: 2.2666666666666666


浴衣着て　線香花火　じっと見る!!!
var: 0.711111111111111


花火大会　ビール片手に　空を飲む!!!
var: 0.17777777777777773


夜空咲く　忘れた頃に　もう一発これは最高傑作です。
var: 0.7111111111111109


夜空咲く　猫も驚く　花火かな😀
var: 0.5


夜空咲き　恋の火花も　散りにけり😀
var: 1.3888888888888888


夜空咲く　ポップコーンにて　待ちぼうけ😀
var: 0.0



## 49. トークン化
以下の文章（夏目漱石の『吾輩は猫である』の冒頭部分）のトークン数を計測せよ。

**吾輩は猫である。名前はまだ無い。**

**どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。**

In [ ]:
import tiktoken

text = '''吾輩は猫である。名前はまだ無い。

どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。\
吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。\
この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。\
ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。\
この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。\
のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。\
どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。
'''

enc = tiktoken.get_encoding('cl100k_base')
tokens = enc.encode(text)
print(f'Number of tokens: {len(tokens)}')

Number of tokens: 496
